# Developing and Using Cornserve with Jupyter Notebooks

An application is composed of `Task`s. `Cornserve` has provided some builtin `Task`s, and here is an example to create a VLM `Task`.

In [3]:
from cornserve_tasklib.task.composite.llm import MLLMTask
from cornserve_tasklib.task.unit.encoder import Modality

mllm = MLLMTask(
    model_id="Qwen/Qwen2-VL-7B-Instruct",
    modalities=[Modality.IMAGE],
)

You can deploy this task to the cluster so that you can later invoke it. To deploy, you need to create a `CornserveClient`.

In [ ]:
from cornserve.frontend import CornserveClient

cornserve = CornserveClient(url="localhost:30080")

Connected to Cornserve gateway at localhost:30080


Error in keep_alive: 1 validation error for TaskResponse
  Invalid JSON: EOF while parsing a value at line 1 column 0 [type=json_invalid, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid


Before deploying your app, deploy the task library so tasks/descriptors classes are available.

In [6]:
cornserve.deploy_tasklib()

TaskResponse(status=200, content='Tasklib deployed: unit=EncoderTask, DecodeLLMUnitTask, LLMUnitTask, PrefillLLMUnitTask; descriptors=EricDescriptor, DecodeVLLMDescriptor, PrefillVLLMDescriptor, VLLMDescriptor; composite=DisaggregatedMLLMTask, MLLMTask')

Now, you can deploy the task on Cornserve with `cornserve.deploy`.

This is going to take a few minutes due to vLLM's startup time. With the `cornserve` CLI on the commandline, you will also get live outputs from each unit task executor.

In [ ]:
cornserve.deploy(mllm)

The builtin `MLLMTask` is compatible with the OpenAI Chat Completion API.

In [ ]:
from cornserve_tasklib.task.unit.llm import OpenAIChatCompletionRequest

mllm_input = OpenAIChatCompletionRequest.model_validate(
    {
        "model": "Qwen/Qwen2-VL-7B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is this image about?"},
                    {
                        "type": "image_url",
                        "image_url": {"url": "https://picsum.photos/seed/random/200/300"},
                    },
                ],
            },
        ],
    }
)

In [5]:
async for chunk in await mllm(mllm_input):
    if chunk.choices:
        print(chunk.choices[0].delta.content, end="")

INFO 2025-07-13 21:45:40,999 [cornserve.task.base:709] Dispatching tasks to http://localhost:30080/tasks/invoke


The image shows a dog wrapped in a blanket. The blanket appears to be plaid or checkered, with a mix of colors including beige, pink, and green. The dog is sitting on the ground, and the blanket is wrapped around its body, covering its head and part of its back. The background suggests an outdoor setting, possibly a forest or a park, with trees and greenery visible. The overall scene gives a cozy and warm impression, as if the dog is being kept warm in the cold weather.

A more structured way of interacting with the Cornserve structure is by writing *apps*. You can find more examples in [our repository](https://github.com/cornserve-ai/cornserve/tree/master/examples).

Finally, to clean up, let's teardown the task that we deployed.

In [6]:
cornserve.teardown(mllm)

TaskResponse(status=200, content='Tasks declared not used')

The client basically maintains a websocket connection with the Cornserve gateway, and disconnecting will also result in tearing down all the tasks you deployed.

In [7]:
cornserve.close()

Closed connection to Cornserve gateway.
Closed keep-alive thread.
